In [1]:
! pip install langchain langchain-community openai tiktoken pinecone-client langchain_pinecone unstructured pdfminer==20191125 pdfminer.six==20221105 pillow_heif unstructured_inference youtube-transcript-api pytube sentence-transformers

python(1045) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
514381.08s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 14.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pillow-10.4.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.0 kB)
INF

In [5]:
! pip install python-dotenv

python(12998) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
515036.11s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [18]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, WebBaseLoader, YoutubeLoader, DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
# from google.colab import userdata
from pinecone import Pinecone
from openai import OpenAI
import numpy as np
import tiktoken
from dotenv import load_dotenv
import os

load_dotenv('.env.local')

pinecone_api_key = os.environ['PINECONE_API_KEY']

openai_api_key = os.environ['OPENAI_KEY']

# Initialize Open AI

In [19]:
embeddings = OpenAIEmbeddings()
embed_model = "text-embedding-3-small"
openai_client = OpenAI()

ValidationError: 1 validation error for OpenAIEmbeddings
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'cli...20, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

# Initialize text splitter

In [ ]:
tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text, 
        disallowed_special=()
    )
    return len(tokens)

text_splitter =RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100,
    length_function=tiktoken_len,
    seperators=["\n\n", "\n", " ", ""]
)


TypeError: TextSplitter.__init__() got an unexpected keyword argument 'seperators'

# Understanding Embeddings

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    # Call the OpenAI API to get the embedding for the text
    response = openai_client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

def cosine_similarity_between_words(sentence1, sentence2):
    # Get embeddings for both words
    embedding1 = np.array(get_embedding(sentence1))
    embedding2 = np.array(get_embedding(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1,-1)
    embeddin21 = embedding2.reshape(1,-1)

    print("Embedding for Sentence 1:", embedding1)
    print("\nEmbedding for Sentence 2:", embedding2)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]

# Example
sentence1 = "I like walking to the park"
sentence2 = "I like walking to the office"

similarity = cosine_similarity_between_words(sentence1, sentence2)
print(f"\n\nCosine similarity between '{sentence1}' and '{sentence2}': '{similarity:4f}'")

# Load YouTube Video and get Transcript

In [ ]:
# load in a YouTube video's transcript
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=tEzs3VHyBDM", add_video_info=True)
data = loader.load()

# print(data)

texts = text_splitter.split_documents(data)
print(texts)

NameError: name 'text_splitter' is not defined

# Initialize Pinecone

In [ ]:
vectorstore = PineconeVectorStore(index_name='OpenAI-o1-demo', embedding=embeddings)

index_name = 'OpenAI-o1-demo'
namespace = 'Building-OpenAI-o1'

# Insert data into Pinecone
#### Documentation: https://docs.pinecone.io/integrations/langchain#key-concepts

In [ ]:
for document in texts:
    print('\n\n\n\n----')
    print(document.metadata, document.page_content)
    print('\n\n\n\n----')

vectorstore_from_texts = PineconeVectorStore.from_texts([f"Source: {t.metadata['source']}, Title: {t.metadata['title']} \n\nContent: {t.page_content}" for t in texts], embeddings, index_name=index_name, namespace=namespace)


# Perform RAG

In [ ]:
from pinecone import Pinecone